In [1]:
from data_utils import preprocessDataset
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import numpy as np

In [2]:
dataset = 'credit'
train_dataset, test_dataset_original = preprocessDataset(f'/media/data_dump/Mohit/facct23_samplebias_data/run_1/{dataset}/raw/betaDatasets/imbalance_1.0_1.0', f'/media/data_dump/Mohit/facct23_samplebias_data/run_1/{dataset}/raw/original_test.csv', dataset)

In [3]:
from sklearn.neural_network import MLPClassifier
import math
from torch import nn
import torch
from sklearn.metrics import accuracy_score

In [4]:
data_shape = train_dataset.features.shape

In [5]:
data_shape

(798, 59)

In [6]:
from skorch import NeuralNetClassifier, NeuralNet
import skorch

In [7]:
class MyModule(nn.Module):
    def __init__(self, data_shape, nonlin=nn.ReLU()):
        super().__init__()
        # -1 for sensitive attribute removal (group-blind training)
        num_units = math.ceil((2*data_shape[0])/(data_shape[1] - 1))
        self.dense0 = nn.Linear(data_shape[1] - 1, num_units)
        self.nonlin = nonlin
        #self.dropout = nn.Dropout(0.5)
        self.output = nn.Linear(num_units, 2)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, data, sample_weight=1, **kwargs):
        X = data
        X = self.nonlin(self.dense0(X))
        X = self.softmax(self.output(X))
        return X

class MyNet(NeuralNet):
    def __init__(self, *args, criterion__reduce=False, **kwargs):
        # make sure to set reduce=False in your criterion, since we need the loss
        # for each sample so that it can be weighted
        super().__init__(*args, criterion__reduce=criterion__reduce, **kwargs)

    def get_loss(self, y_pred, y_true, X, *args, **kwargs):
        # override get_loss to use the sample_weight from X
        loss_unreduced = super().get_loss(y_pred, y_true, X, *args, **kwargs)
        sample_weight = skorch.utils.to_tensor(X['sample_weight'], device=self.device)
        loss_reduced = (sample_weight * loss_unreduced).mean()
        return loss_reduced

In [8]:
net = MyNet(
    MyModule(data_shape),
    criterion=nn.NLLLoss,
    optimizer=torch.optim.Adam,
    train_split=None,
    max_epochs=100,
    lr=0.01,
    # Shuffle training data on each epoch
    iterator_train__shuffle=True,
    device=0
)

In [9]:
net_2 = NeuralNetClassifier(
    MyModule(data_shape),
    criterion=nn.NLLLoss,
    optimizer=torch.optim.Adam,
    train_split=None,
    max_epochs=100,
    lr=0.01,
    # Shuffle training data on each epoch
    iterator_train__shuffle=True,
    device=0
)

In [10]:
X = train_dataset.features[:,:-1]
y = train_dataset.labels.ravel().astype(int)
X = X.astype(np.float32)

In [11]:
# X = {'data': X,
#     'sample_weight': np.ones_like(y).astype(np.float32)}

In [12]:
net_2.fit(X, y)

  epoch    train_loss     dur
-------  ------------  ------
      1        0.6120  0.0289
      2        0.5311  0.0233
      3        0.4846  0.0253
      4        0.4595  0.0234
      5        0.4497  0.0235
      6        0.4362  0.0327
      7        0.4206  0.0324
      8        0.4120  0.0260
      9        0.3993  0.0238
     10        0.3853  0.0240
     11        0.3704  0.0237
     12        0.3569  0.0234
     13        0.3436  0.0240
     14        0.3311  0.0226
     15        0.3127  0.0242
     16        0.2981  0.0230
     17        0.2874  0.0236
     18        0.2780  0.0227
     19        0.2758  0.0234
     20        0.2433  0.0241
     21        0.2330  0.0238
     22        0.2241  0.0233
     23        0.2140  0.0241
     24        0.1993  0.0231
     25        0.1932  0.0235
     26        0.1826  0.0233
     27        0.1752  0.0240
     28        0.1691  0.0233
     29        0.1602  0.0235
     30        0.1518  0.0233
     31        0.1426  0.0240
     32   

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=MyModule(
    (dense0): Linear(in_features=58, out_features=28, bias=True)
    (nonlin): ReLU()
    (output): Linear(in_features=28, out_features=2, bias=True)
    (softmax): Softmax(dim=-1)
  ),
)

In [13]:
net_2.predict(X)[:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [13]:
test_dataset = test_dataset_original
X_test = test_dataset.features[:,:-1]
y_test = test_dataset.labels.ravel().astype(int)
X_test = X_test.astype(np.float32)

X_test = {'data': X_test,
    'sample_weight': np.ones_like(y_test).astype(np.float32)}

In [14]:
accuracy_score(y_test, np.argmax(net.predict(X_test), axis=1))

0.848140770252324

In [15]:
accuracy_score(y, np.argmax(net.predict(X), axis=1))

0.8541210795040116

In [67]:
from sklearn.neural_network import MLPClassifier

In [68]:
cls = MLPClassifier(hidden_layer_sizes=(math.ceil((2*data_shape[0])/(data_shape[1] - 1)),)).fit(X['data'],y)

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.


In [69]:
accuracy_score(y, cls.predict(X['data']))

0.9246071215436642

In [70]:
accuracy_score(y_test, cls.predict(X_test['data']))

0.8310756972111554

In [ ]:
#from folktables import ACSIncome, ACSDataSource

In [8]:
#data_source = ACSDataSource(survey_year='2018', horizon='5-Year', survey='person', #root_dir='/media/data_dump/Mohit/facct23_samplebias_data/large_adult_source/')

In [ ]:
#acs_data = data_source.get_data(download=True)